# SML BONUS ASSIGNMENT

## IMPORT LIBRARIES

In [126]:
import numpy as np
import sklearn
import csv
import os
import cv2
from skimage.feature import hog
from skimage import feature
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes as nb
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd 
from sklearn.decomposition import PCA
import xgboost as xgb
import sklearn.preprocessing as prp
from sklearn.ensemble import RandomForestClassifier
import pickle

### DEFINE PATH OF DATASET

In [2]:
path = "D:\\IIITD\\2nd Sem\\SML\\Assignments\\Bonus\\cse342-542-bonus\\bonus_dataset"

### READ LABELS

In [3]:
def read_label_dict():
    data={}
    with open(path + '\\sml_train.csv') as csvfile:
        reader=csv.reader(csvfile, skipinitialspace=True, quotechar="'")
        for row in reader:
            data[row[0]]=row[1:]
    del data['Id']
    return data

### READ TRAINING DATA

In [4]:
def read_train_data():
    images = []
    filename_list = []
    folder = path + "\\sml_train"
    for filename in os.listdir(folder):
        
        img = cv2.imread(os.path.join(folder,filename))
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            images.append(img)
            filename_list.append(filename)

    return images, filename_list

In [5]:
def read_train_data_hsv():
    images = []
    filename_list = []
    folder = path + "\\sml_train"
    for filename in os.listdir(folder):
        
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        if img is not None:
            images.append(img)
            filename_list.append(filename)

    return images

In [6]:
def read_train_data_gray():
    images = []
    filename_list = []
    folder = path + "\\sml_train"
    for filename in os.listdir(folder):
        
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            images.append(img)
            filename_list.append(filename)

    return images

### READ TEST DATA

In [7]:
def read_test_data():
    images = []
    filename_list =[]
    folder = path + "/sml_test"
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            images.append(img)
            filename_list.append(filename)
    return images,filename_list

In [8]:
def read_test_data_hsv():
    images = []
    filename_list =[]
    folder = path + "/sml_test"
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        if img is not None:
            images.append(img)
            filename_list.append(filename)
    return images

In [9]:
def read_test_data_gray():
    images = []
    filename_list =[]
    folder = path + "/sml_test"
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            images.append(img)
            filename_list.append(filename)
    return images

### GETTING DATA READY

In [10]:
label_dict = read_label_dict()
train_data, filename_list = read_train_data()
train_data_gray = read_train_data_gray()
test_data, filename_list_test = read_test_data()
test_data_gray = read_test_data_gray()
train_data_hsv = read_train_data_hsv() 
test_data_hsv = read_test_data_hsv()

In [12]:
label_list = []
for i in filename_list:
    label_list.append(label_dict[i])

### FEATURE EXTRACTION

#### HSV PIXEL INTENSITY

In [101]:
hsv_features = np.zeros((len(train_data),12288))
train_data_hsv = np.array(train_data_hsv)
for i in range(0,len(train_data_hsv)):
    hsv_features[i] = np.ravel(train_data_hsv[i])

In [102]:
hsv_features_test = np.zeros((len(test_data),12288))
test_data_hsv = np.array(test_data_hsv)
for i in range(0,len(test_data_hsv)):
    hsv_features_test[i] = np.ravel(test_data_hsv[i])

#### RGB PIXEL INTENSITY

In [103]:
intensity_features = np.zeros((len(train_data),12288))
train_data = np.array(train_data)
for i in range(0,len(train_data)):
    intensity_features[i] = np.ravel(train_data[i])
#    intensity_features[i] = prp.scale(intensity_features[i])
#intensity_features = np.ravel(train_data)

In [104]:
intensity_features_test = np.zeros((len(test_data),12288))
test_data = np.array(test_data)
for i in range(0,len(test_data)):
    intensity_features_test[i] = np.ravel(test_data[i])
    #intensity_features_test[i] = prp.scale(intensity_features_test[i])
#intensity_features = np.ravel(train_data)

#### HOG

In [105]:
discrip = []
for i in range(0,len(train_data)):
    fd = hog(train_data[i],orientations=8,pixels_per_cell=(16,16),cells_per_block=(4,4),block_norm='L2')
    #img = train_data[i].reshape(64,-1)
    #lbp = feature.local_binary_pattern(img,256, 3, method="uniform")
    #hist = np.histogram(lbp , bins=128)
    #discriptor = train_data[i].flatten()
    #discriptor = PCA(n_components=64).fit_transform(discriptor)
    
    #final_fd = np.concatenate((fd,discriptor))
    
    discrip.append(fd)


In [106]:
discrip_test = []
for i in range(0,len(test_data)):
    fd = hog(test_data[i],orientations=8,pixels_per_cell=(16,16),cells_per_block=(4,4),block_norm='L2')
#     img = test_data[i].reshape(64,-1)
#     lbp = feature.local_binary_pattern(img,256, 3, method="uniform")
#     hist = np.histogram(lbp , bins=128)
#     discriptor = hist[0]
#     final_fd = np.concatenate((fd,discriptor))
    discrip_test.append(fd)

### PRE-PROCESSING

In [107]:
hsv_features = prp.quantile_transform(hsv_features)
#hsv_features = prp.scale(hsv_features)

In [108]:
hsv_features_test = prp.quantile_transform(hsv_features_test)
#hsv_features_test = prp.scale(hsv_features_test)

In [109]:
intensity_features = prp.quantile_transform(intensity_features)
#intensity_features = prp.scale(intensity_features)

In [110]:
intensity_features_test = prp.quantile_transform(intensity_features_test)
#intensity_features_test = prp.scale(intensity_features_test)

In [111]:
discrip = prp.quantile_transform(discrip)
#discrip = prp.scale(discrip)

In [112]:
discrip_test = prp.quantile_transform(discrip_test)
#discrip_test = prp.scale(discrip_test)

### DIMENSIONALITY REDUCTION

In [113]:
pca = PCA(n_components=40).fit(hsv_features)
hsv_features_pca = pca.transform(hsv_features)
hsv_features_pca_test = pca.transform(hsv_features_test)

In [114]:
pca = PCA(n_components=40).fit(intensity_features)
intensity_features_pca = pca.transform(intensity_features)
intensity_features_pca_test = pca.transform(intensity_features_test)

In [115]:
pca = PCA(n_components=40).fit(discrip)
train_hog_pca = pca.transform(discrip)
test_hog_pca = pca.transform(discrip_test)

In [18]:
# lbp_dis_lis = []
# for i in range(0,len(train_data)):
#     #img = train_data_gray[i].reshape(64,-1)
#     lbp = feature.local_binary_pattern(train_data_gray[i],64, 3, method="uniform")
# #    hist = np.histogram(lbp , bins=128)
#  #   discriptor = hist[0]
#     lbp = lbp.flatten()
#     lbp_dis_lis.append(lbp)

In [97]:
# lbp_dis_lis_test = []
# for i in range(0,len(test_data)):
#     #img = train_data_gray[i].reshape(64,-1)
#     lbp = feature.local_binary_pattern(test_data_gray[i],64, 3, method="uniform")
# #    hist = np.histogram(lbp , bins=128)
#  #   discriptor = hist[0]
#     lbp = lbp.flatten()
#     lbp_dis_lis_test.append(lbp)

In [98]:
# pca = PCA(n_components=30).fit(lbp_dis_lis)
# train_lbp_pca = pca.transform(lbp_dis_lis)

# test_lbp_pca = pca.transform(lbp_dis_lis_test)

In [116]:
final_train = np.concatenate((train_hog_pca,intensity_features_pca,hsv_features_pca), axis=1)

In [117]:
final_test = np.concatenate((test_hog_pca,intensity_features_pca_test,hsv_features_pca_test),axis = 1)

(1000, 120)

#### TRAIN TEST SPLIT

In [118]:
x_train, x_test, y_train, y_test = train_test_split(
         final_train, label_list, test_size=0.3)

## CLASSIFIERS

### LOGISTIC REGRESSION

In [37]:
logis_clf = LogisticRegression().fit(final_train , label_list)
logis_acc = logis_clf.score(final_train , label_list)
pred = logis_clf.predict(final_test)
print('accuracy for logistic regression' , logis_acc)

C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy for logistic regression 0.3462


(1000,)


### GAUSSIAN NAIVE BAYES

In [38]:
naive_clf = nb.GaussianNB().fit(final_train , label_list)
naive_acc = naive_clf.score(final_train , label_list)
pred = naive_clf.predict(final_test)
print('accuracy for naive bayes' , naive_acc)

C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy for naive bayes 0.368


### SUPPORT VECTOR MACHINE

In [40]:
svm_clf = SVC(gamma="auto" , decision_function_shape='ovo').fit(x_train,y_train)
svm_acc = svm_clf.score(x_test,y_test)
print('accuracy for support vector machine' , svm_acc)

C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy for support vector machine 0.043666666666666666


In [122]:
linearSVC_clf = LinearSVC().fit(discrip , label_list)
pred = linearSVC_clf.predict(discrip_test)
svm_acc = linearSVC_clf.score(discrip , label_list)
print('accuracy for support vector machine' , svm_acc)

C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy for support vector machine 0.3382


In [43]:
svm_acc = linearSVC_clf.score(x_test,y_test)
print('accuracy for support vector machine' , svm_acc)

accuracy for support vector machine 0.2806666666666667


### XGBOOST

In [ ]:
xgb_model = xgb.XGBClassifier(learning_rate=0.1,n_estimators=500,max_depth=8,min_child_weight=3,gamma=0.2,
                              subsample=0.6,colsample_bytree=1.0,objective='binary:logistic',nthread=4,scale_pos_weight=1,
                              seed=27).fit(final_train , label_list)
pred = xgb_model.pred(final_test)

C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\MitTal\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### RANDOM FOREST

In [119]:
RF_clf = RandomForestClassifier(n_estimators= 500, random_state=42).fit(x_train,y_train)
print(RF_clf.score(x_test , y_test))

C:\Users\MitTal\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


0.456


In [120]:
RF_clf = RandomForestClassifier(n_estimators= 500, random_state=42).fit(final_train , label_list)
pred = RF_clf.predict(final_test)


C:\Users\MitTal\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [127]:
file = "model_randomForest"
pickle.dump(RF_clf , open(file , "wb"))

In [121]:
filename_list_test = np.array(filename_list_test)
combined = np.vstack((filename_list_test, pred)).T

In [125]:
np.savetxt("output.csv",combined , delimiter=',' , header='Id,Category',fmt='%s')